<a href="https://colab.research.google.com/github/yuli-05/AoGCourseCF/blob/master/Caso_de_uso.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Cargar archivo de datos
datos = pd.read_csv("/content/datos_request.csv")

# Renombrar columnas para facilitar su uso
datos.rename(columns={
    "No. de Solicitud": "Numero_Solicitud",
    "Nombre": "Nombre",
    "Apellido": "Apellido",
    "Tiempo estimado (min)": "Tiempo_Estimado"
}, inplace=True)

# Validación: Pedir un nuevo tiempo si hay valores negativos o mayores a 120 minutos
for index, row in datos.iterrows():
    while row["Tiempo_Estimado"] > 120 or row["Tiempo_Estimado"] < 0:
        print(f" Error: {row['Nombre']} {row['Apellido']} ingresó un tiempo de {row['Tiempo_Estimado']} min (fuera del rango permitido).")
        nuevo_tiempo = int(input(f"Ingrese un nuevo tiempo válido (entre 0 y 120 min) para {row['Nombre']} {row['Apellido']}: "))
        datos.at[index, "Tiempo_Estimado"] = nuevo_tiempo  # Se actualiza el valor corregido
        row["Tiempo_Estimado"] = nuevo_tiempo  # Para que se valide correctamente en el bucle


# Ordenar por tiempo estimado
#Se ordenan las solicitudes de menor a mayor tiempo estimado.
#Objetivo: Atender primero las solicitudes con tiempos menores para reducir la espera acumulada.
# Ordenar primero por tiempo estimado y luego por número de solicitud
datos = datos.sort_values(by=["Tiempo_Estimado", "Numero_Solicitud"])


#tiempo_actual: Lleva el control del tiempo en que cada solicitud empieza a ser atendida.
#tiempo_espera_total: Suma de los tiempos de espera de todos los ciudadanos.
#resultados: Lista donde se almacenan los datos procesados.
tiempo_actual = 0
tiempo_espera_total = 0
resultados = []

#Cálculo del tiempo de espera:

#La primera persona tiene tiempo_espera = 0 porque es atendida inmediatamente.
#Cada persona siguiente espera el tiempo acumulado de las citas previas.

for index, row in datos.iterrows():
    tiempo_espera = tiempo_actual  # Tiempo de espera antes de ser atendido
    tiempo_espera_total += tiempo_espera  # Acumulamos tiempos de espera

    resultados.append({
        "Numero_Solicitud": row["Numero_Solicitud"],
        "Nombre": row["Nombre"] + " " + row["Apellido"],
        "Tiempo_Estimado": row["Tiempo_Estimado"],
        "Tiempo_Espera": tiempo_espera
    })

    tiempo_actual += row["Tiempo_Estimado"]  # Se actualiza el tiempo de atención

# Convertir resultados en DataFrame y mostrar
resultados_df = pd.DataFrame(resultados)

# Muestra la tabla con las citas ordenadas y los tiempos de espera de cada usuario.
print("\nResultados de la programación de citas:")
print(resultados_df)

# Guardar el DataFrame ordenado en un nuevo archivo CSV
resultados_df.to_csv("datos_ordenados.csv", index=False)

print("\nLos datos ordenados han sido guardados en 'datos_ordenados.csv'.")







Resultados de la programación de citas:
     Numero_Solicitud             Nombre  Tiempo_Estimado  Tiempo_Espera
0                  33       Martín Pérez               10              0
1                  43    Carmen Palacios               10             10
2                 166     Pablo Quintero               11             20
3                  62      Pedro Morales               13             31
4                  71       Alfredo Nava               13             44
..                ...                ...              ...            ...
195                22  Mariana Maldonado              117          12512
196                40   Francisco Campos              117          12629
197                50   Cristina Navarro              118          12746
198               110  Mónica Villanueva              118          12864
199               174     Eduardo Rivera              119          12982

[200 rows x 4 columns]

Los datos ordenados han sido guardados en 'datos_ordenados